# Model Predictive Control Rocket Soft-Landing Problem

This notebook is a walkthough of the ALTRO solver for the MPC Rocket Soft-Landing Problem (i.e. the general control challenge of landing a rocket at rest at a designated landing site). In the simplest case, we treat the rocket as a sphere landing on a flat surface with no atmosphere. This approximation is reasonable when the rocket is near the ground and entering the final part of the descent.

**This files focuses on the MPC problem and comparing the result to other solvers. For the simple (non-MPC) application and set-up explanati, please visit the <code>RocketLandingSimplified.ipynb</code> file. Thanks!**

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

 Activating environment at `C:\Users\Daniel N\.julia\dev\altro-mpc-icra2021\benchmarks\rocket_landing\Project.toml`


## MPC Problem Outline

For the MPC application, we must fulfill two critical steps:
1. Solve the optimal trajectory and apply the required controls (usually a short horizon)
2. Use the prior trajectory to warm-start the next solve

In this Jupyter notebook, we will:
1. Set up the rocket soft-landing problem 
2. Solve the initial trajectory optimization problem (cold-start)
3. Execute the control (and propagate forward in time)
4. Warm-Start the next iteration
5. Solve the next shorter horizon trajectory optimization problem
6. Repeat steps 3-5 until arrival at the landing location

# Step 1: Set-Up the Rocket Soft-Landing Problem

## Load Required Packages

The <code>RobotDynamics</code> package enables us to define the rocket model and dynamics. The <code>TrajectoryOptimization</code> and <code>Altro</code> set-up and solve the trajectory optimization problem. <code>StaticArrays</code> and <code>LinearAlgebra</code> are required to set-up the appropriate data types.

Next, <code>Plots, MeshCat, TrajOptPlots, FileIO,</code> and <code>MeshIO</code> packages are used to visualize the results. Note that the plotting sections will take significantly longer to run than the optimization. 

Lastly, <code>DifferentialEquations</code> integrates the dynamics forward as a way to show that the dynamics constraints are satisfied.

Loading all the packages, especially for plotting and the differential equations integrator can be the longest part of the whole notebook.

In [2]:
using RobotDynamics
import RobotDynamics: dynamics
using Rotations
using Altro
using TrajectoryOptimization
using StaticArrays, LinearAlgebra
println("Trajectory Optimization Packages Ready")

using Plots
pyplot()
println("Base Plotting Packages Ready")

using MeshCat, TrajOptPlots
using FileIO, MeshIO
println("3D Plotting Packages Ready")

import DifferentialEquations.solve, DifferentialEquations.ODEProblem, DifferentialEquations.Tsit5
using BenchmarkTools
println("Differential Equations Ready")

Trajectory Optimization Packages Ready
Base Plotting Packages Ready
3D Plotting Packages Ready
Differential Equations Ready


## Detail the Rocket Dynamics

For the simplified rocket problem, we choose to ignore the attitude dynamics. Applying newton's second law or by forming a Lagrangian, we arrive at the following sparse linear system

$$\frac{dx}{dt} = Ax + Bu + g$$

Where $A = \begin{bmatrix}0 & I\\ 0 & 0 \end{bmatrix}$, $B = \begin{bmatrix}0 \\ \frac{1}{m} I\end{bmatrix}$, and $G = \begin{bmatrix} 0 \\ -g \end{bmatrix}$. For the 3D case, these are $(6 \times 6)$, $(6 \times 3)$, and $(6 \times 1)$ matricies, respectively. 

In [3]:
const mass = 10.0
const grav = SA[0.0; 0.0; -9.81]

model = LinearModel([zeros(3,3) I; zeros(3, 6)], [zeros(3,3); (1/mass) * I], vec([zeros(3, 1); grav]))

LinearModel{6,3,Float64,15}(SizedArray{Tuple{6,6},Float64,2,2}[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], SizedArray{Tuple{6,3},Float64,2,2}[[0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.1 0.0; 0.0 0.0 0.1]], SizedArray{Tuple{6},Float64,1,1}[[0.0, 0.0, 0.0, 0.0, 0.0, -9.81]], Float64[], 0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], true)

## Instantiate the Model and Define the Discretization

Feel free to choose <code>N</code> and <code>tf</code> based on your needs.

In [4]:
n, m = size(model)
# n is the size of the states ([x, y, z, vx, vy, vz])
# m is the size of the control thrust ([Tx, Ty, Tz]])

# Trajectory Discretization
N = 251
tf = 10.
dt = tf/(N-1)

model # (Simply for information) Display the struct as the cell output 

LinearModel{6,3,Float64,15}(SizedArray{Tuple{6,6},Float64,2,2}[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], SizedArray{Tuple{6,3},Float64,2,2}[[0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.1 0.0; 0.0 0.0 0.1]], SizedArray{Tuple{6},Float64,1,1}[[0.0, 0.0, 0.0, 0.0, 0.0, -9.81]], Float64[], 0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], true)

## Specify the Initial and Final Conditions

Choose the initial and final points. Note that these must be full state vectors ($[x; y; z; v_x; v_y; v_z]$). The initial point is generally the rocket's current state. The final point is generally the rocket's landing stop. Note that the final velocity of the rocket should **always** be $[0.0; 0.0; 0.0]$ to have a **soft** landing.

In [5]:
# Initial and Final Conditions
x0 = @SVector [4.0, 2.0, 20.0, -3.0, 2.0, -5.0] # Start at a 20 m altitude with an ~15 m/s plummet
xf = @SVector zeros(n);  # Swing pendulum up and end at rest

## Set-Up the LQR Objective Function

For this problem, we choose an LQR Objective Function. One can add waypoints or change the weights to craft different optimal trajectories. See http://roboticexplorationlab.org/TrajectoryOptimization.jl/stable/costfunctions.html for more information.

In [6]:
Q = 1.0e-2 * Diagonal(@SVector ones(n))
Qf = 100.0 * Diagonal(@SVector ones(n))
R = 1.0e-1 * Diagonal(@SVector ones(m))
obj = LQRObjective(Q,R,Qf,xf,N)

Objective

## Define the Constraints

For the simple rocket soft-landing problem, we will include 4 constraints:
1. Landing Site Designation ($x[t_f] = x_f$)
2. Crash Prevention Constraint ($z > 0$)
3. Maximum Thrust Constraint ($0 < ||u|| < u_{max}$), which is a simplification of a max thrust constraint
4. Maximum Thurst Angle Constraint ($||[u_x; u_y]|| ≤ \alpha \ u_z$), which is a simplification of a max thrust angle constraint 

For details on the implemented constraint types, please check out http://roboticexplorationlab.org/TrajectoryOptimization.jl/stable/constraints.html.

In [7]:
# Create Empty ConstraintList
conSet = ConstraintList(n,m,N)

# Key Constants
ground_level = 0.0 # Crash Prevention Constraint
theta = 7 # deg. Max Angle Constraint
perWeightMax = 2
u_bnd = mass * norm(grav[3]) * perWeightMax
println("Maximum Thrust is $u_bnd")


# Goal Constraint that the rocket must reach the landing site.
goal = GoalConstraint(xf)
add_constraint!(conSet, goal, N)

# Bounds
bnd = BoundConstraint(n,m, x_min=[-Inf, -Inf, ground_level,
                                  -Inf, -Inf, -Inf])

# Include the max thrust constraint
maxT = NormConstraint(n, m, u_bnd, TrajectoryOptimization.SecondOrderCone(), :control)
add_constraint!(conSet, maxT, 1:N-1)

# Include the max thrust angle constraint
function getAlpha(theta, deg = true)
    if deg
        return tand(theta)
    end
    return tan(theta)
end


maxTAalpha = getAlpha(theta)
println("α = $maxTAalpha")

Maximum Thrust is 196.20000000000002


0.1227845609029046

## Stitch together the objective and constraints

See http://roboticexplorationlab.org/TrajectoryOptimization.jl/stable/creating_problems.html for more information.

In [8]:
# Package the objective and constraints into a "problem" type
prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet);

## Initialize the Trajectory

We choose to initialize the trajectory at a hover (i.e. the rocket trajectory will initially only counter gravity). Note that this is **not** a warm start.

In [9]:
u0 = grav # controls that would nominally hover
U0 = [u0 for k = 1:N-1] # vector of the small controls
initial_controls!(prob, U0)
rollout!(prob);

## Select the solver options

For more information, please visit https://github.com/RoboticExplorationLab/Altro.jl/blob/master/README.md for more details on the solver options.

Using the Altro package, we can now specify parameters and solve the problem.

In [10]:
opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0,
    verbose = 1,
    projected_newton = false
)

SolverOptions{Float64}
  constraint_tolerance: Float64 1.0e-6
  cost_tolerance: Float64 0.0001
  cost_tolerance_intermediate: Float64 0.01
  gradient_tolerance: Float64 10.0
  gradient_tolerance_intermediate: Float64 1.0
  iterations_inner: Int64 300
  dJ_counter_limit: Int64 10
  square_root: Bool false
  line_search_lower_bound: Float64 1.0e-8
  line_search_upper_bound: Float64 10.0
  iterations_linesearch: Int64 20
  max_cost_value: Float64 1.0e8
  max_state_value: Float64 1.0e8
  max_control_value: Float64 1.0e8
  static_bp: Bool true
  save_S: Bool false
  bp_reg: Bool false
  bp_reg_initial: Float64 0.0
  bp_reg_increase_factor: Float64 1.6
  bp_reg_max: Float64 1.0e8
  bp_reg_min: Float64 1.0e-8
  bp_reg_type: Symbol control
  bp_reg_fp: Float64 10.0
  penalty_initial: Float64 1.0
  penalty_scaling: Float64 10.0
  active_set_tolerance_al: Float64 0.001
  dual_max: Float64 NaN
  penalty_max: Float64 NaN
  iterations_outer: Int64 30
  kickout_max_penalty: Bool false
  reset_duals:

# Step 2: Solve the initial trajectory optimization problem

Show summary is set to <code>true</code> to print the solver speed. In general the solve will take about 23 iterations and 20-40 ms to complete. The constraint violation should be around $10^{-4}$ or better.

In [13]:
altro = ALTROSolver(prob, opts);

In [ ]:
set_options!(altro, show_summary=false)
b = benchmark_solve!(altro)

Now we can solve it and show the summary

In [12]:
set_options!(altro, show_summary=true)
solve!(altro);

iter  total  c_max       cost      info                                              
-------------------------------------------------------------------------------------
1     2       0.95036258  5295.419 
2     4       0.86706575  5300.01  
3     6       0.44223298  5323.038 
4     8       0.04169586  5342.115 
5     10      0.0004296   5343.077 
iter  total  c_max       cost      info                                              
-------------------------------------------------------------------------------------
6     11      4.4678e-07  5343.078 



SOLVE COMPLETED
 solved using the ALTRO Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 11
    Solve Time: 18020.156001 (ms)

  Covergence
    Terminal Cost: 5343.077292240792
    Terminal dJ: 8.638355781592509
    Terminal gradient: 0.008593850048396239
    Terminal constraint violation: 4.467832226710475e-7
    Solve Status: SOLVE_SUCCEEDED


Access the Trajectory

In [ ]:
X = states(altro)
U = controls(altro);

## Step 3: Integrate the Dynamics to Verify the Dynamics Constraint

In Matlab, we would use ODE 45, but in the <code>DifferentialEquations.jl</code> package we can either (1) just let the <code>solve</code> method figure it out or (2) specify a solver such as <code>Tsit5</code>.

In [ ]:
num_timesteps = 3

get_controls(t) = U[min(Int(floor(t/dt)) + 1, size(U,1))]
f(y,p,t) = dynamics(model, y, get_controls(t))

states_y0 = x0
tspan = (0.0, dt * num_timesteps)
controls_p0 = get_controls(tspan[1])

# Use the DifferentialEquations.jl Package
ode_prob = ODEProblem(f, states_y0, tspan, controls_p0)
ode_sol = solve(ode_prob, Tsit5(), reltol=1e-10, abstol=1e-10);

## Step 4: Warm-Start the next iteration

In [ ]:
# warm_start_states = X[num_timesteps:end]
warm_start_controls = U[num_timesteps:end]
x0 = ode_sol.u[end]

N = size(warm_start_controls, 1) + 1
tf = tf - dt * num_timesteps
dt = tf / (N - 1)

## Step 5 & 6: Solve the Problem and Repeat

Now that we've seen each step in detail, let's put it all into a function and repeat it until landing.

In [ ]:
function diffEq_Model(model::Rocket, U, dt)
    get_controls(t) = U[min(Int(floor(t/dt)) + 1, size(U,1))]
    f(y,p,t) = dynamics(model, y, get_controls(t))

    states_y0 = x0
    tspan = (0.0, dt * num_timesteps)
    controls_p0 = get_controls(tspan[1])

    # Use the DifferentialEquations.jl Package
    ode_prob = ODEProblem(f, states_y0, tspan, controls_p0)
    ode_sol = solve(ode_prob, Tsit5(), reltol=1e-10, abstol=1e-10);

    return ode_sol.u[end]
end

function MPC(model::Rocket, solver, x0, U_init, N, tf)
    
#     println("Setting Up Problem")
    # Reset the objective function
    obj = LQRObjective(Q,R,Qf,xf,N)
    
    # Create Empty ConstraintList
    conSet = ConstraintList(n,m,N)

    # Bounds
    ground_level = 0.0 # Crash Prevention Constraint
    umax = 11 * model.mass # Simplified Max Thrust Constraint
    theta = 20 # deg. Max Angle Constraint
    ulateral_max = umax * sind(theta)

    # To ignore constraints 2 and 3, use the code below
    # bnd = BoundConstraint(n,m, x_min=[-Inf, -Inf, ground_level,
    #                                   -Inf, -Inf, -Inf])

    # To include constraints 2 and 3, use the code below
    bnd = BoundConstraint(n,m, x_min=[-Inf, -Inf, ground_level,
                                      -Inf, -Inf, -Inf],
                               u_min=[-ulateral_max, -ulateral_max, 0   ],
                               u_max=[ ulateral_max,  ulateral_max, umax])
    add_constraint!(conSet, bnd, 1:N-1)

    # Goal Constraint that the rocket must reach the landing site.
    goal = GoalConstraint(xf)
    add_constraint!(conSet, goal, N)
    
    # Package the objective and constraints into a "problem" type
    prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet);
    
#     println("Initial Controls")
    # Set the current controls (ideally warm-started)
    initial_controls!(prob, U_init)
    rollout!(prob);
    
#     println("Solving")
    # Solve the new trajectory
    a_s = ALTROSolver(prob, opts)
    solve!(a_s);
    set_options!(a_s, show_summary=false)
    b = benchmark_solve!(a_s)
    display(b)
    
#     println("Accessing")
    # Access the states and controls
    X = states(a_s)
    U = controls(a_s)
    
#     println("Propogate System")
    # Use the dynamics model (and any disturbances) and propogate forward in time
    x0 = diffEq_Model(model, U, dt)
    
#     println("Warm-Start Next Iteration")
    U_new = U[num_timesteps:end]
    N = size(U_new, 1) + 1
    tf = tf - dt * num_timesteps
    
    # Warm-Start the Next Iteration
    return x0, U_new, N, tf
    
end

In [ ]:
a = MPC(model, ALTROSolver, x0, warm_start_controls, N, tf);

Execute the MPC forward until landed

In [ ]:
completion_threshold = 1

opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0
)

saved_states = []

while norm(x0 - xf) > completion_threshold
    x0, warm_start_controls, N, tf = MPC(model, ALTROSolver, x0, warm_start_controls, N, tf)
    
    println("\nCurrently at time $tf remaining and $(norm(x0 - xf)) away.")
    println("$N Steps at $tf s left. Hence, dt = $(tf / (N-1))")
    
    # If the timespan is too low (e.g. due to disturbances)
    tf = max(0.5, tf)
    
    push!(saved_states, x0)
end

In [ ]:
xs = [x[1] for x in saved_states]
zs = [x[3] for x in saved_states];

x_orig = [x[1] for x in X]
z_orig = [x[3] for x in X];

## Plot the trajectory!

In [ ]:
titleText = "Rocket Soft-Landing Trajectory and MPC Path"
filename = "RocketGroundThrustBound_BarelyFeasible3"

plt_x = plot(x_orig, z_orig, label = "Original Planned Trajectory")
plot!(xs, zs, label = "MPC Tracked Trajectory")
xlabel!("x")
ylabel!("z (height)")
title!(titleText)

In [ ]:
xmin = minimum(xs)
xmax = maximum(xs)

zmin = minimum(zs)
zmax = maximum(zs)

@gif for i in 1:size(xs, 1)
    plt = plot(xs[1:i], zs[1:i], label = "Trajectory", legend = :topleft)
    xlabel!("x")
    ylabel!("z (height)")
    xlims!(xmin,xmax)
    ylims!(zmin,zmax)
    title!(titleText)
end

In [ ]:
plt_ux = plot(uxs, label = "Ux Control")
hline!([-ulateral_max, ulateral_max], linecolor = :grey, linestyle = :dash,
                label = "Max Lateral Thrust")
# xlabel!("time (s)")
ylabel!("control (N)")
title!("Controls over Time")

plt_uz = plot(uzs, label = "Uz Control")
hline!([umax], linecolor = :grey, linestyle = :dash, label = "Max Thrust")
xlabel!("time (s)")
ylabel!("control (N)")

plt_u = plot(plt_ux, plt_uz, layout = (2, 1))

Below is a GIF of the trajectory. This will take several seconds to generate.

## Lastly, let's visualize the results in 3D with Meshcat

To do this, we need to provide a Mesh to <code>MeshCat</code>. This is due with <code>FileIO</code> and <code>MeshIO</code>.

In [ ]:
vis = Visualizer() # Creates a MeshCat Window
render(vis) # Renders the window below

In [ ]:
function TrajOptPlots._set_mesh!(vis, model::Rocket)
    obj = joinpath(@__DIR__, "SimpleRocket.obj")
    quad_scaling = 0.085
    robot_obj = FileIO.load(obj)
    robot_obj.vertices .*= quad_scaling
    mat = MeshPhongMaterial(color=colorant"lightgrey")
    setobject!(vis["geom"], robot_obj, mat)
end

TrajOptPlots.set_mesh!(vis, model)

In [ ]:
Base.position(::Rocket, x::StaticVector) = SA[x[1], x[2], x[3]]
RobotDynamics.orientation(::Rocket, x::StaticVector) = UnitQuaternion(0,0,0,1)

visualize!(vis, altro);

## Integrate the Dynamics to Verify the Dynamics Constraint

In Matlab, we would use ODE 45, but in the <code>DifferentialEquations.jl</code> package we can either (1) just let the <code>solve</code> method figure it out or (2) specify a solver such as <code>Tsit5</code>.

The controls change over time and are discretized. Below, a zero-order hold and a first-order hold are implemented for comparison. **We used a zero-order hold in the optimization so a zero-order hold should be used here.**

In [ ]:
get_controls_zero_order_hold(t) = U[min(Int(floor(t/dt)) + 1, size(U,1))]
f(y,p,t) = dynamics(model, y, get_controls_zero_order_hold(t))

states_y0 = x0
tspan = (0.0, tf)
controls_p0 = get_controls_zero_order_hold(tspan[1])

ode_prob_zoh = ODEProblem(f, states_y0, tspan, controls_p0)
@btime solve(ode_prob_zoh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt)
ode_sol_zoh = solve(ode_prob_zoh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt);

In [ ]:
function get_controls_first_order_hold(t)
    # dt and U are both global variables
    tn = t/dt
    
    t_below = Int(max(1, floor(tn)))
    t_above = Int(min(size(U, 1), ceil(tn)))
    
    if t_below >= t_above
        return U[t_below]
    end
    
    slope = (U[t_above] - U[t_below])
    return slope * (tn - t_below) + U[t_below]

end


f(y,p,t) = dynamics(model, y, get_controls_first_order_hold(t))

states_y0 = x0
tspan = (0.0, tf)
controls_p0 = get_controls_first_order_hold(tspan[1])

ode_prob_foh = ODEProblem(f, states_y0, tspan, controls_p0)
@btime solve(ode_prob_foh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt)
ode_sol_foh = solve(ode_prob_foh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt);

In [ ]:
err_zoh = norm.(ode_sol_zoh.u - X[1:(Int(floor(tspan[2]/dt))+1)])
err_foh = norm.(ode_sol_foh.u - X[1:(Int(floor(tspan[2]/dt))+1)])

plot(ode_sol_zoh.t, err_zoh, yaxis = :log, label = "Zero-Order Hold")
plot!(ode_sol_foh.t, err_foh, yaxis = :log, label = "First-Order Hold")
xlabel!("Time (s)")
ylabel!("Dynamics Error")
title!("Comparison in State Vector expected from ALTRO Solver \n to Integrated Dynamics Model")